In [46]:
from graphics import *
import random
import numpy as np


In [47]:
win = GraphWin("My Window", 600,600)
win.setBackground(color_rgb(255,255,255))


In [45]:
def line(x1,y1,x2,y2):
    """
    create a line between pt(x1,y1) and pt(x2,y2) and return it
    """
    ln = Line(Point(x1,y1),Point(x2,y2))
    ln.setOutline(color_rgb(0,0,0))
    ln.setWidth(2)
    
    return ln


In [32]:
#border lines
border1 = line(50,50,550,50)
border1.draw(win)

border2 = line(550,50,550,550)
border2.draw(win)

border3 = line(50,550,550,550)
border3.draw(win)

border4 = line(50,50,50,550)
border4.draw(win)

#lines horizontal
ln1 = line(50,150,550,150)
ln1.draw(win)

ln2 = line(50,250,550,250)
ln2.draw(win)

ln3 = line(50,350,550,350)
ln3.draw(win)

ln4 = line(50,450,550,450)
ln4.draw(win)

#lines vertical

lnv1 = line(150,50,150,550)
lnv1.draw(win)

lnv2 = line(250,50,250,550)
lnv2.draw(win)

lnv3 = line(350,50,350,550)
lnv3.draw(win)

lnv4 = line(450,50,450,550)
lnv4.draw(win)

#create goal, obstacle and hole

rect_goal = Rectangle(Point(450,50), Point(550,150))
rect_goal.setFill(color_rgb(0,255,0))
rect_goal.draw(win)

rect_hole = Rectangle(Point(450,150), Point(550,250))
rect_hole.setFill(color_rgb(255,0,0))
rect_hole.draw(win)

rect_obs1 = Rectangle(Point(150,350), Point(250,450))
rect_obs1.setFill(color_rgb(0,0,0))
rect_obs1.draw(win)

rect_obs2 = Rectangle(Point(150,250), Point(250,350))
rect_obs2.setFill(color_rgb(0,0,0))
rect_obs2.draw(win)

Rectangle(Point(150.0, 250.0), Point(250.0, 350.0))

In [33]:
agent = Circle(Point(100,500), 25)
agent.setFill(color_rgb(0,0,255))
agent.draw(win)

Circle(Point(100.0, 500.0), 25)

In [34]:
def up(agent):
    val = agent.getCenter()
    
    if (val.getY()) == 100 or (val.getY() == 500 and val.getX() == 200) :
        pass
    else :
        agent.move(0,-100)
        
    
def down(agent):
    val = agent.getCenter()
    
    if val.getY() == 500 or (val.getX() == 200 and val.getY() == 200):
        pass
    else :
        agent.move(0,100)

def right(agent):
    val = agent.getCenter()
    
    if val.getX() == 500 or (val.getX() == 100 and (val.getY() == 300 or val.getY() == 400)):
        pass
    else :
        agent.move(100,0)

def left(agent):
    val = agent.getCenter()
    
    if val.getX() == 100 or (val.getX() == 300 or (val.getY() == 300 or val.getY() == 400)):
        pass
    else :
        agent.move(-100,0)
        
def action(agent,val):
    if val == 0:
        up(agent)
    elif val == 1:
        down(agent)
    elif val == 2:
        right(agent)
    else :
        left(agent)

In [35]:
reward = np.array([[-1,-1,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-1,-1,-1,-100],
                   [-1,-1,-1,-1,100]])


In [36]:
def rewards(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    
    return reward[j][i]

def observation(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100) 
    
    return [i,j]

def env_close():
    win.close()

def reset(agent):
    agent.move(-400,400)

def set_agent(agent,obs):
    pos = agent.getCenter()
    
    pos = [pos.getX(), pos.getY()]
    
    refer_pos = [(obs[0]+1) * 100, (500 - (obs[1]*100))]
    
    
    agent.move(refer_pos[0]-pos[0], refer_pos[1] - pos[1])

In [37]:
value = np.array([[.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,.0],
                  [.0,.0,.0,.0,0.0],
                  [.0,.0,.0,.0,100.0]])

In [38]:
policy = np.array([[0,0,0,0,2],
                   [2,0,0,0,2],
                   [0,0,0,0,2],
                   [0,0,0,0,2],
                   [0,0,3,0,0]])


In [39]:
number_visit = np.zeros([5,5])

In [40]:
episode = 4000
max_episode_length = 100
learning_rate = .01
discount_factor = 0.9


In [41]:
for j in range(episode):  
    
    Experience_replay = []
    Return = 0
    incremental_discount_prev = 1
    incremental_discount_next = 1
    
    for i in range(max_episode_length):
            
        state = observation(agent)
        
        number_visit[state[0],state[1]] += 1
        
        if state[0]== 4 and state[1] == 4:
            reset(agent)
            break
        
        act = policy[state[0],state[1]]
        
        #add probability to each action
        if act == 0:
            act = np.random.choice([0,1,2,3],p=[0.8,0,0.1,0.1])
        elif act == 1:
            act = np.random.choice([0,1,2,3],p=[0,0.8,0.1,0.1])
        elif act == 2:
            act = np.random.choice([0,1,2,3],p=[0.1,0.1,0.8,0])
        elif act == 3:
            act = np.random.choice([0,1,2,3],p=[0.1,0.1,0, 0.8])
            
        #act on the grid world
        action(agent,act)
        
        incremental_experience = [state[0],state[1],-Return,incremental_discount_next]
        
        Experience_replay.append(incremental_experience)
        
        incremental_discount_next = incremental_discount_next * discount_factor
    
        Reward = rewards(agent) * incremental_discount_prev
        Return = Return + Reward
        
        incremental_discount_prev = incremental_discount_next
        
    
 
    for exp in Experience_replay:
        value[exp[0],exp[1]] = value[exp[0],exp[1]] + (learning_rate * (((Return+ exp[2])/ exp[3] )- value[exp[0],exp[1]]))

In [42]:
value

array([[ 32.10932362,  38.08337746,  45.09729054,  51.78402568,
         59.44651071],
       [ 33.78896065,   0.        ,   0.        ,  60.45124271,
         70.78624878],
       [ 40.59801026,  48.36702909,  57.53598403,  68.31264736,
         82.75893402],
       [ 10.16527002,  20.15453   ,  40.98900032,  71.381497  ,
         96.48843539],
       [ -0.55087824,  -4.84051644, -12.15203172,  51.77508651,
        100.        ]])